<a href="https://colab.research.google.com/github/chiragsanadhya/Text_wizard-/blob/main/Text_wizard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
!pip install python-pptx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 19.7 MB/s eta 0:00:00


In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.0 MB/s eta 0:00:00


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer


model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
from docx import Document
from pptx import Presentation
import PyPDF2

In [ ]:
def generate_summary_from_text_chunks(text_chunks, tokenizer, model, max_length=150, num_beams=4):
    summaries = []
    for chunk in text_chunks:
        inputs = tokenizer(chunk, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(inputs["input_ids"], num_beams=num_beams, max_length=max_length, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)
    return '\n\n'.join(summaries)

def generate_summary_from_file(file_path, tokenizer, model, max_length=150, num_beams=4, chunk_size=4):
    with open(file_path, 'r') as file:
        file_content_txt = file.read()
    # Split the text into chunks of four paragraphs each
    paragraphs = file_content_txt.split('\n\n')
    text_chunks = [paragraphs[i:i+chunk_size] for i in range(0, len(paragraphs), chunk_size)]
    return generate_summary_from_text_chunks(text_chunks, tokenizer, model, max_length, num_beams)

def generate_summary_from_docx(file_path, tokenizer, model, max_length=150, num_beams=4, chunk_size=4):
    doc = Document(file_path)
    doc_text = ""
    for paragraph in doc.paragraphs:
        doc_text += paragraph.text + "\n"
    # Split the text into chunks of four paragraphs each
    paragraphs = doc_text.split('\n\n')
    text_chunks = [paragraphs[i:i+chunk_size] for i in range(0, len(paragraphs), chunk_size)]
    return generate_summary_from_text_chunks(text_chunks, tokenizer, model, max_length, num_beams)

def generate_summary_from_pptx(ppt_file, tokenizer, model, max_length=150, num_beams=4, chunk_size=4):
    presentation = Presentation(ppt_file)
    slide_texts = ""
    for slide in presentation.slides:
        for shape in slide.shapes:
            if hasattr(shape, 'text'):
                slide_texts += shape.text + '\n'
    # Split the text into chunks of four paragraphs each
    paragraphs = slide_texts.split('\n\n')
    text_chunks = [paragraphs[i:i+chunk_size] for i in range(0, len(paragraphs), chunk_size)]
    return generate_summary_from_text_chunks(text_chunks, tokenizer, model, max_length, num_beams)

def generate_summary_from_pdf(pdf_file, tokenizer, model, max_length=150, num_beams=4, chunk_size=4):
    with open(pdf_file, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        pdf_text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            pdf_text += page.extract_text()
    # Split the text into chunks of four paragraphs each
    paragraphs = pdf_text.split('\n\n')
    text_chunks = [paragraphs[i:i+chunk_size] for i in range(0, len(paragraphs), chunk_size)]
    return generate_summary_from_text_chunks(text_chunks, tokenizer, model, max_length, num_beams)


In [ ]:
# text_file = '/content/project_experiment.rtf'
# summary = generate_summary_from_file(text_file, tokenizer, model)
# print(summary_TXT)




# docx_file = '/content/project_experiment.docx'
# summary = generate_summary_from_docx(docx_file, tokenizer, model)
# print(summary_DOCX)



pdf_file = '/content/toaz.info-the-hundred-page-machine-learning-book-pr_ae27c30ee81d0fc238f6f47a8b8805f1.pdf'
summary = generate_summary_from_pdf(pdf_file, tokenizer, model)
print(summary)




# ppt_file = '/content/Environmental-Regulations-PP.pptx'
# summary = generate_summary_from_pptx(ppt_file, tokenizer, model)
# print(summary_PDF)

FileNotFoundError: [Errno 2] No such file or directory: '/content/toaz.info-the-hundred-page-machine-learning-book-pr_ae27c30ee81d0fc238f6f47a8b8805f1.pdf'